#### Scores are from model that is trained from start of year, 3 years ago
#### So if today is 9/9/2019, the scores added today should be from model that
#### takes everything done before 1/1/2016 as a trainable loan
#### e.g. a loan's end_d < 1/1/2016

# This notebooks adds scores from all models to eval_loan_info

In [1]:
# steps when adding new score from new model:
#     1) load in test_eval_loan_info_scored.fth
#     2) add model scoring steps into class Model:
#         a) load model
#         b) process base_loan_info in same way model was trained
#     3) score all loans, add column to eval_loan_info
#     4) save over eval_loan_info_scored.fth:
#         a) Maybe make a backup of the previous eval_loan_info_scored.fth?

In [14]:
%%writefile ../../lendingclub/modeling/models.py
'''
Defines the Model class and methods.
Usage:
    Instantiate a model:
        baseline = Model('baseline')
    Get scores for a dataframe:
        scores = baseline.score(df)
'''


import os

import numpy as np
import pandas as pd
from catboost import CatBoostClassifier, CatBoostRegressor
from joblib import load

import j_utils.munging as mg
from lendingclub import config

ppath = config.prj_dir
dpath = config.data_dir
mpath = config.model_dir


class Model():
    '''
    Model class loads appropriate model based on name in constructor
    Also handles preprocessing
    '''
    def __init__(self, name: str):
        self.name = name
        self.basempath = os.path.join(ppath, 'models')
        self.mpath = os.path.join(self.basempath, self.name)
        self.m = None
        self.df = None

    def score(self, df: pd.DataFrame):
        '''
        Given a dataframe (base_loan_info, non imputed or scaled or normalized)
        return scores. Imputation, Scaling, and Normalizing will be handled
        inside this method to match that done at training
        '''
        # baselines and grades
        if self.name == 'baseline':
            return np.random.random(len(df))
        elif self.name in ['A', 'B', 'C', 'D', 'E', 'F', 'G']:
            scores = np.random.random(len(df))
            mask = np.where(df['grade'] == self.name, 0, 1).astype(bool)
            scores[mask] = 0
            return scores
        # logistic regression
        elif self.name == 'logistic_regr':
            self.m = load(os.path.join(self.mpath, 'logistic_regr.joblib'))
            self.proc_data(df)
            preds = self.m.predict_proba(self.df)[:, 0]
            return preds
        # catboost classifier
        elif self.name == 'catboost_clf':
            catboost_clf = CatBoostClassifier()
            catboost_clf.load_model(os.path.join(self.mpath,
                                                 'catboost_clf.cb'))
            self.m = catboost_clf
            self.proc_data(df)
            preds = self.m.predict_proba(self.df)[:, 0]
            return preds
        # catboost classifier
        elif self.name == 'catboost_clf_cbdp':
            catboost_clf = CatBoostClassifier()
            catboost_clf.load_model(os.path.join(self.mpath,
                                                 'catboost_clf.cb'))
            self.m = catboost_clf
            self.proc_data_cb(df)
            preds = self.m.predict_proba(self.df)[:, 0]
            return preds
        # catboost regressor
        elif self.name == 'catboost_regr':
            catboost_regr = CatBoostRegressor()
            catboost_regr.load_model(
                os.path.join(self.mpath, 'catboost_regr.cb'))
            self.m = catboost_regr
            self.proc_data(df)
            preds = self.m.predict(self.df)
            return preds
        # catboost regressor
        elif self.name == 'catboost_regr_cbdp':
            catboost_regr = CatBoostRegressor()
            catboost_regr.load_model(
                os.path.join(self.mpath, 'catboost_regr.cb'))
            self.m = catboost_regr
            self.proc_data_cb(df)
            preds = self.m.predict(self.df)
            return preds
        print('unknown model??')
        return None

    def proc_data_cb(self, df):
        '''
        process data before passing it as input for training.
        For catboost, letting catboost deal with categoricals
        '''
        date_cols = df.select_dtypes('datetime')
        for col in date_cols:
            df[col] = df[col].astype('str')
        self.df = df

    def proc_data(self, df):
        '''
        process data before passing it as input for training.
        Uses the j_utils.munging process data format, and requires loading
        7 pickles of colnames, max/mins,
        fill values, categorical mappings, and numbers needed for normalizing
        '''
        assert 'end_d' not in df, print(
            'must pass base_loan_info, not eval_loan_info')
        self.df = df
        all_train_colnames = load(
            os.path.join(self.mpath, 'all_train_colnames.joblib'))
        max_dict = load(os.path.join(self.mpath, 'max_dict.joblib'))
        min_dict = load(os.path.join(self.mpath, 'min_dict.joblib'))
#         new_null_colnames = load(
#             os.path.join(self.mpath, 'new_null_colnames.joblib'))
        fill_dict = load(os.path.join(self.mpath, 'fill_dict.joblib'))
        cats_dict = load(os.path.join(self.mpath, 'cats_dict.joblib'))
        norm_dict = load(os.path.join(self.mpath, 'norm_dict.joblib'))
        self.df = mg.val_test_proc(df, all_train_colnames, max_dict, min_dict,
                                   fill_dict, cats_dict, norm_dict)

Overwriting ../../lendingclub/modeling/models.py


In [9]:
from lendingclub.modeling import models

In [5]:
import numpy as np

In [8]:
np.random.random(10)

array([0.12032497, 0.91106567, 0.60612363, 0.36499969, 0.31175309,
       0.27892237, 0.90934623, 0.79988992, 0.9680308 , 0.54441858])

In [26]:
base_loan_info = pd.read_feather(os.path.join(dpath, 'base_loan_info.fth'))
# eval_loan_info = pd.read_feather(os.path.join(dpath, 'eval_loan_info.fth'))

# baselines

In [5]:
for model in ['baseline', 'A', 'B', 'C', 'D', 'E', 'F', 'G']:
    m = Model(model)
    eval_loan_info['{0}_score'.format(model)] = m.score(base_loan_info)

In [6]:
eval_loan_info.head()

,end_d,issue_d,maturity_paid,maturity_time,maturity_time_stat_adj,maturity_paid_stat_adj,rem_to_be_paid,roi_simple,target_loose,target_strict,loan_status,id,grade,int_rate,term,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.2,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.3,0.31,0.32,0.33,0.34,0.35,baseline_score,A_score,B_score,C_score,D_score,E_score,F_score,G_score
0,2011-10-01,2009-08-01,1.000000,1.000000,1.0,1.0,0.000000,1.173214,0,0,paid,54734,B,0.1189,36,0.095902,0.081254,0.066865,0.052729,0.038841,0.025198,0.011792,-0.001379,-0.014321,-0.027038,-0.039534,-0.051815,-0.063884,-0.075745,-0.087403,-0.098861,-0.110124,-0.121195,-0.132078,-0.142776,-0.153294,-0.163634,-0.173800,-0.183796,-0.193624,-0.203289,-0.212792,-0.222138,-0.231329,-0.240368,-0.249259,0.989311,0.000000,0.812243,0.0,0.0,0.000000,0.000000,0.0
1,2010-03-01,2008-07-01,1.000000,1.000000,1.0,1.0,0.000000,1.207769,0,0,paid,55521,F,0.1608,36,0.137690,0.124301,0.111112,0.098121,0.085324,0.072718,0.060299,0.048065,0.036013,0.024139,0.012441,0.000915,-0.010441,-0.021630,-0.032654,-0.043517,-0.054222,-0.064770,-0.075164,-0.085407,-0.095501,-0.105449,-0.115253,-0.124916,-0.134439,-0.143825,-0.153077,-0.162196,-0.171184,-0.180044,-0.188778,0.588934,0.000000,0.000000,0.0,0.0,0.000000,0.759759,0.0
2,2018-06-01,2016-08-01,1.000000,0.944444,1.0,1.0,0.000000,1.353502,0,0,paid,55716,E,0.2499,36,0.274963,0.259472,0.244227,0.229225,0.214461,0.199930,0.185629,0.171554,0.157699,0.144063,0.130640,0.117426,0.104419,0.091615,0.079009,0.066600,0.054382,0.042352,0.030509,0.018847,0.007364,-0.003943,-0.015077,-0.026041,-0.036838,-0.047472,-0.057944,-0.068258,-0.078416,-0.088421,-0.098276,0.379164,0.000000,0.000000,0.0,0.0,0.175012,0.000000,0.0
3,2011-06-01,2008-05-01,1.000000,1.000000,1.0,1.0,0.000000,1.173648,0,0,paid,55742,B,0.1071,36,0.087770,0.071643,0.055847,0.040373,0.025214,0.010362,-0.004190,-0.018448,-0.032420,-0.046112,-0.059532,-0.072685,-0.085577,-0.098215,-0.110604,-0.122751,-0.134660,-0.146337,-0.157788,-0.169018,-0.180031,-0.190833,-0.201429,-0.211822,-0.222018,-0.232022,-0.241836,-0.251467,-0.260917,-0.270191,-0.279292,0.140504,0.000000,0.247415,0.0,0.0,0.000000,0.000000,0.0
4,2018-04-01,2016-01-01,0.583209,1.000000,1.0,1.0,3677.225098,0.643185,1,1,charged_off,56121,A,0.0649,36,-0.385032,-0.390533,-0.395965,-0.401330,-0.406627,-0.411859,-0.417025,-0.422128,-0.427167,-0.432145,-0.437061,-0.441916,-0.446713,-0.451450,-0.456130,-0.460753,-0.465320,-0.469831,-0.474288,-0.478691,-0.483042,-0.487340,-0.491586,-0.495782,-0.499928,-0.504025,-0.508072,-0.512072,-0.516025,-0.519931,-0.523792,0.829386,0.162706,0.000000,0.0,0.0,0.000000,0.000000,0.0


In [7]:
eval_loan_info.to_feather(os.path.join(dpath, 'eval_loan_info_scored.fth'))

# Logistic Regr

In [25]:
eval_loan_info = pd.read_feather(os.path.join(dpath, 'eval_loan_info_scored.fth'))

In [26]:
m = Model('logistic_regr')
eval_loan_info['logistic_regr_score'] = m.score(base_loan_info)

In [27]:
m.score(base_loan_info)

array([0.6048379 , 0.28425111, 0.24071816, ..., 0.7158517 , 0.55519109,
       0.54293517])

In [25]:
eval_loan_info.head()

,end_d,issue_d,maturity_paid,maturity_time,maturity_time_stat_adj,maturity_paid_stat_adj,rem_to_be_paid,roi_simple,target_loose,target_strict,loan_status,id,grade,int_rate,term,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.2,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.3,0.31,0.32,0.33,0.34,0.35,baseline_score,A_score,B_score,C_score,D_score,E_score,F_score,G_score,logistic_regr_score
0,2011-10-01,2009-08-01,1.000000,1.000000,1.0,1.0,0.000000,1.173214,0,0,paid,54734,B,0.1189,36,0.095902,0.081254,0.066865,0.052729,0.038841,0.025198,0.011792,-0.001379,-0.014321,-0.027038,-0.039534,-0.051815,-0.063884,-0.075745,-0.087403,-0.098861,-0.110124,-0.121195,-0.132078,-0.142776,-0.153294,-0.163634,-0.173800,-0.183796,-0.193624,-0.203289,-0.212792,-0.222138,-0.231329,-0.240368,-0.249259,0.989311,0.000000,0.812243,0.0,0.0,0.000000,0.000000,0.0,0.604838
1,2010-03-01,2008-07-01,1.000000,1.000000,1.0,1.0,0.000000,1.207769,0,0,paid,55521,F,0.1608,36,0.137690,0.124301,0.111112,0.098121,0.085324,0.072718,0.060299,0.048065,0.036013,0.024139,0.012441,0.000915,-0.010441,-0.021630,-0.032654,-0.043517,-0.054222,-0.064770,-0.075164,-0.085407,-0.095501,-0.105449,-0.115253,-0.124916,-0.134439,-0.143825,-0.153077,-0.162196,-0.171184,-0.180044,-0.188778,0.588934,0.000000,0.000000,0.0,0.0,0.000000,0.759759,0.0,0.284251
2,2018-06-01,2016-08-01,1.000000,0.944444,1.0,1.0,0.000000,1.353502,0,0,paid,55716,E,0.2499,36,0.274963,0.259472,0.244227,0.229225,0.214461,0.199930,0.185629,0.171554,0.157699,0.144063,0.130640,0.117426,0.104419,0.091615,0.079009,0.066600,0.054382,0.042352,0.030509,0.018847,0.007364,-0.003943,-0.015077,-0.026041,-0.036838,-0.047472,-0.057944,-0.068258,-0.078416,-0.088421,-0.098276,0.379164,0.000000,0.000000,0.0,0.0,0.175012,0.000000,0.0,0.240723
3,2011-06-01,2008-05-01,1.000000,1.000000,1.0,1.0,0.000000,1.173648,0,0,paid,55742,B,0.1071,36,0.087770,0.071643,0.055847,0.040373,0.025214,0.010362,-0.004190,-0.018448,-0.032420,-0.046112,-0.059532,-0.072685,-0.085577,-0.098215,-0.110604,-0.122751,-0.134660,-0.146337,-0.157788,-0.169018,-0.180031,-0.190833,-0.201429,-0.211822,-0.222018,-0.232022,-0.241836,-0.251467,-0.260917,-0.270191,-0.279292,0.140504,0.000000,0.247415,0.0,0.0,0.000000,0.000000,0.0,0.570257
4,2018-04-01,2016-01-01,0.583209,1.000000,1.0,1.0,3677.225098,0.643185,1,1,charged_off,56121,A,0.0649,36,-0.385032,-0.390533,-0.395965,-0.401330,-0.406627,-0.411859,-0.417025,-0.422128,-0.427167,-0.432145,-0.437061,-0.441916,-0.446713,-0.451450,-0.456130,-0.460753,-0.465320,-0.469831,-0.474288,-0.478691,-0.483042,-0.487340,-0.491586,-0.495782,-0.499928,-0.504025,-0.508072,-0.512072,-0.516025,-0.519931,-0.523792,0.829386,0.162706,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.825928


In [27]:
eval_loan_info.to_feather(os.path.join(dpath, 'eval_loan_info_scored.fth'))

# Catboost_clf

In [19]:
eval_loan_info = pd.read_feather(os.path.join(dpath, 'eval_loan_info_scored.fth'))
eval_loan_info.head()

,end_d,issue_d,maturity_paid,maturity_time,maturity_time_stat_adj,maturity_paid_stat_adj,rem_to_be_paid,roi_simple,target_loose,target_strict,loan_status,id,grade,int_rate,term,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.2,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.3,0.31,0.32,0.33,0.34,0.35,baseline_score,A_score,B_score,C_score,D_score,E_score,F_score,G_score,logistic_regr_score,catboost_clf_score
0,2011-10-01,2009-08-01,1.000000,1.000000,1.0,1.0,0.000000,1.173214,0,0,paid,54734,B,0.1189,36,0.095902,0.081254,0.066865,0.052729,0.038841,0.025198,0.011792,-0.001379,-0.014321,-0.027038,-0.039534,-0.051815,-0.063884,-0.075745,-0.087403,-0.098861,-0.110124,-0.121195,-0.132078,-0.142776,-0.153294,-0.163634,-0.173800,-0.183796,-0.193624,-0.203289,-0.212792,-0.222138,-0.231329,-0.240368,-0.249259,0.989311,0.000000,0.812243,0.0,0.0,0.000000,0.000000,0.0,0.604838,0.586115
1,2010-03-01,2008-07-01,1.000000,1.000000,1.0,1.0,0.000000,1.207769,0,0,paid,55521,F,0.1608,36,0.137690,0.124301,0.111112,0.098121,0.085324,0.072718,0.060299,0.048065,0.036013,0.024139,0.012441,0.000915,-0.010441,-0.021630,-0.032654,-0.043517,-0.054222,-0.064770,-0.075164,-0.085407,-0.095501,-0.105449,-0.115253,-0.124916,-0.134439,-0.143825,-0.153077,-0.162196,-0.171184,-0.180044,-0.188778,0.588934,0.000000,0.000000,0.0,0.0,0.000000,0.759759,0.0,0.284251,0.469791
2,2018-06-01,2016-08-01,1.000000,0.944444,1.0,1.0,0.000000,1.353502,0,0,paid,55716,E,0.2499,36,0.274963,0.259472,0.244227,0.229225,0.214461,0.199930,0.185629,0.171554,0.157699,0.144063,0.130640,0.117426,0.104419,0.091615,0.079009,0.066600,0.054382,0.042352,0.030509,0.018847,0.007364,-0.003943,-0.015077,-0.026041,-0.036838,-0.047472,-0.057944,-0.068258,-0.078416,-0.088421,-0.098276,0.379164,0.000000,0.000000,0.0,0.0,0.175012,0.000000,0.0,0.240723,0.142538
3,2011-06-01,2008-05-01,1.000000,1.000000,1.0,1.0,0.000000,1.173648,0,0,paid,55742,B,0.1071,36,0.087770,0.071643,0.055847,0.040373,0.025214,0.010362,-0.004190,-0.018448,-0.032420,-0.046112,-0.059532,-0.072685,-0.085577,-0.098215,-0.110604,-0.122751,-0.134660,-0.146337,-0.157788,-0.169018,-0.180031,-0.190833,-0.201429,-0.211822,-0.222018,-0.232022,-0.241836,-0.251467,-0.260917,-0.270191,-0.279292,0.140504,0.000000,0.247415,0.0,0.0,0.000000,0.000000,0.0,0.570257,0.608114
4,2018-04-01,2016-01-01,0.583209,1.000000,1.0,1.0,3677.225098,0.643185,1,1,charged_off,56121,A,0.0649,36,-0.385032,-0.390533,-0.395965,-0.401330,-0.406627,-0.411859,-0.417025,-0.422128,-0.427167,-0.432145,-0.437061,-0.441916,-0.446713,-0.451450,-0.456130,-0.460753,-0.465320,-0.469831,-0.474288,-0.478691,-0.483042,-0.487340,-0.491586,-0.495782,-0.499928,-0.504025,-0.508072,-0.512072,-0.516025,-0.519931,-0.523792,0.829386,0.162706,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.825928,0.662058


In [20]:
m = Model('catboost_clf')
eval_loan_info['catboost_clf_score'] = m.score(base_loan_info)

In [21]:
eval_loan_info.head()

,end_d,issue_d,maturity_paid,maturity_time,maturity_time_stat_adj,maturity_paid_stat_adj,rem_to_be_paid,roi_simple,target_loose,target_strict,loan_status,id,grade,int_rate,term,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.2,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.3,0.31,0.32,0.33,0.34,0.35,baseline_score,A_score,B_score,C_score,D_score,E_score,F_score,G_score,logistic_regr_score,catboost_clf_score
0,2011-10-01,2009-08-01,1.000000,1.000000,1.0,1.0,0.000000,1.173214,0,0,paid,54734,B,0.1189,36,0.095902,0.081254,0.066865,0.052729,0.038841,0.025198,0.011792,-0.001379,-0.014321,-0.027038,-0.039534,-0.051815,-0.063884,-0.075745,-0.087403,-0.098861,-0.110124,-0.121195,-0.132078,-0.142776,-0.153294,-0.163634,-0.173800,-0.183796,-0.193624,-0.203289,-0.212792,-0.222138,-0.231329,-0.240368,-0.249259,0.989311,0.000000,0.812243,0.0,0.0,0.000000,0.000000,0.0,0.604838,0.895790
1,2010-03-01,2008-07-01,1.000000,1.000000,1.0,1.0,0.000000,1.207769,0,0,paid,55521,F,0.1608,36,0.137690,0.124301,0.111112,0.098121,0.085324,0.072718,0.060299,0.048065,0.036013,0.024139,0.012441,0.000915,-0.010441,-0.021630,-0.032654,-0.043517,-0.054222,-0.064770,-0.075164,-0.085407,-0.095501,-0.105449,-0.115253,-0.124916,-0.134439,-0.143825,-0.153077,-0.162196,-0.171184,-0.180044,-0.188778,0.588934,0.000000,0.000000,0.0,0.0,0.000000,0.759759,0.0,0.284251,0.700437
2,2018-06-01,2016-08-01,1.000000,0.944444,1.0,1.0,0.000000,1.353502,0,0,paid,55716,E,0.2499,36,0.274963,0.259472,0.244227,0.229225,0.214461,0.199930,0.185629,0.171554,0.157699,0.144063,0.130640,0.117426,0.104419,0.091615,0.079009,0.066600,0.054382,0.042352,0.030509,0.018847,0.007364,-0.003943,-0.015077,-0.026041,-0.036838,-0.047472,-0.057944,-0.068258,-0.078416,-0.088421,-0.098276,0.379164,0.000000,0.000000,0.0,0.0,0.175012,0.000000,0.0,0.240723,0.811421
3,2011-06-01,2008-05-01,1.000000,1.000000,1.0,1.0,0.000000,1.173648,0,0,paid,55742,B,0.1071,36,0.087770,0.071643,0.055847,0.040373,0.025214,0.010362,-0.004190,-0.018448,-0.032420,-0.046112,-0.059532,-0.072685,-0.085577,-0.098215,-0.110604,-0.122751,-0.134660,-0.146337,-0.157788,-0.169018,-0.180031,-0.190833,-0.201429,-0.211822,-0.222018,-0.232022,-0.241836,-0.251467,-0.260917,-0.270191,-0.279292,0.140504,0.000000,0.247415,0.0,0.0,0.000000,0.000000,0.0,0.570257,0.860447
4,2018-04-01,2016-01-01,0.583209,1.000000,1.0,1.0,3677.225098,0.643185,1,1,charged_off,56121,A,0.0649,36,-0.385032,-0.390533,-0.395965,-0.401330,-0.406627,-0.411859,-0.417025,-0.422128,-0.427167,-0.432145,-0.437061,-0.441916,-0.446713,-0.451450,-0.456130,-0.460753,-0.465320,-0.469831,-0.474288,-0.478691,-0.483042,-0.487340,-0.491586,-0.495782,-0.499928,-0.504025,-0.508072,-0.512072,-0.516025,-0.519931,-0.523792,0.829386,0.162706,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.825928,0.852738


In [22]:
eval_loan_info.to_feather(os.path.join(dpath, 'eval_loan_info_scored.fth'))

# Catboost_regr

In [5]:
eval_loan_info = pd.read_feather(os.path.join(dpath, 'eval_loan_info_scored.fth'))
eval_loan_info.head()

,end_d,issue_d,maturity_paid,maturity_time,maturity_time_stat_adj,maturity_paid_stat_adj,rem_to_be_paid,roi_simple,target_loose,target_strict,loan_status,id,grade,int_rate,term,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.2,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.3,0.31,0.32,0.33,0.34,0.35,baseline_score,A_score,B_score,C_score,D_score,E_score,F_score,G_score,logistic_regr_score,catboost_clf_score
0,2011-10-01,2009-08-01,1.000000,1.000000,1.0,1.0,0.000000,1.173214,0,0,paid,54734,B,0.1189,36,0.095902,0.081254,0.066865,0.052729,0.038841,0.025198,0.011792,-0.001379,-0.014321,-0.027038,-0.039534,-0.051815,-0.063884,-0.075745,-0.087403,-0.098861,-0.110124,-0.121195,-0.132078,-0.142776,-0.153294,-0.163634,-0.173800,-0.183796,-0.193624,-0.203289,-0.212792,-0.222138,-0.231329,-0.240368,-0.249259,0.989311,0.000000,0.812243,0.0,0.0,0.000000,0.000000,0.0,0.604838,0.895790
1,2010-03-01,2008-07-01,1.000000,1.000000,1.0,1.0,0.000000,1.207769,0,0,paid,55521,F,0.1608,36,0.137690,0.124301,0.111112,0.098121,0.085324,0.072718,0.060299,0.048065,0.036013,0.024139,0.012441,0.000915,-0.010441,-0.021630,-0.032654,-0.043517,-0.054222,-0.064770,-0.075164,-0.085407,-0.095501,-0.105449,-0.115253,-0.124916,-0.134439,-0.143825,-0.153077,-0.162196,-0.171184,-0.180044,-0.188778,0.588934,0.000000,0.000000,0.0,0.0,0.000000,0.759759,0.0,0.284251,0.700437
2,2018-06-01,2016-08-01,1.000000,0.944444,1.0,1.0,0.000000,1.353502,0,0,paid,55716,E,0.2499,36,0.274963,0.259472,0.244227,0.229225,0.214461,0.199930,0.185629,0.171554,0.157699,0.144063,0.130640,0.117426,0.104419,0.091615,0.079009,0.066600,0.054382,0.042352,0.030509,0.018847,0.007364,-0.003943,-0.015077,-0.026041,-0.036838,-0.047472,-0.057944,-0.068258,-0.078416,-0.088421,-0.098276,0.379164,0.000000,0.000000,0.0,0.0,0.175012,0.000000,0.0,0.240723,0.811421
3,2011-06-01,2008-05-01,1.000000,1.000000,1.0,1.0,0.000000,1.173648,0,0,paid,55742,B,0.1071,36,0.087770,0.071643,0.055847,0.040373,0.025214,0.010362,-0.004190,-0.018448,-0.032420,-0.046112,-0.059532,-0.072685,-0.085577,-0.098215,-0.110604,-0.122751,-0.134660,-0.146337,-0.157788,-0.169018,-0.180031,-0.190833,-0.201429,-0.211822,-0.222018,-0.232022,-0.241836,-0.251467,-0.260917,-0.270191,-0.279292,0.140504,0.000000,0.247415,0.0,0.0,0.000000,0.000000,0.0,0.570257,0.860447
4,2018-04-01,2016-01-01,0.583209,1.000000,1.0,1.0,3677.225098,0.643185,1,1,charged_off,56121,A,0.0649,36,-0.385032,-0.390533,-0.395965,-0.401330,-0.406627,-0.411859,-0.417025,-0.422128,-0.427167,-0.432145,-0.437061,-0.441916,-0.446713,-0.451450,-0.456130,-0.460753,-0.465320,-0.469831,-0.474288,-0.478691,-0.483042,-0.487340,-0.491586,-0.495782,-0.499928,-0.504025,-0.508072,-0.512072,-0.516025,-0.519931,-0.523792,0.829386,0.162706,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.825928,0.852738


In [7]:
m = Model('catboost_regr')
eval_loan_info['catboost_regr_score'] = m.score(base_loan_info)

In [8]:
eval_loan_info.head()

,end_d,issue_d,maturity_paid,maturity_time,maturity_time_stat_adj,maturity_paid_stat_adj,rem_to_be_paid,roi_simple,target_loose,target_strict,loan_status,id,grade,int_rate,term,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.2,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.3,0.31,0.32,0.33,0.34,0.35,baseline_score,A_score,B_score,C_score,D_score,E_score,F_score,G_score,logistic_regr_score,catboost_clf_score,catboost_regr_score
0,2011-10-01,2009-08-01,1.000000,1.000000,1.0,1.0,0.000000,1.173214,0,0,paid,54734,B,0.1189,36,0.095902,0.081254,0.066865,0.052729,0.038841,0.025198,0.011792,-0.001379,-0.014321,-0.027038,-0.039534,-0.051815,-0.063884,-0.075745,-0.087403,-0.098861,-0.110124,-0.121195,-0.132078,-0.142776,-0.153294,-0.163634,-0.173800,-0.183796,-0.193624,-0.203289,-0.212792,-0.222138,-0.231329,-0.240368,-0.249259,0.989311,0.000000,0.812243,0.0,0.0,0.000000,0.000000,0.0,0.604838,0.895790,0.004597
1,2010-03-01,2008-07-01,1.000000,1.000000,1.0,1.0,0.000000,1.207769,0,0,paid,55521,F,0.1608,36,0.137690,0.124301,0.111112,0.098121,0.085324,0.072718,0.060299,0.048065,0.036013,0.024139,0.012441,0.000915,-0.010441,-0.021630,-0.032654,-0.043517,-0.054222,-0.064770,-0.075164,-0.085407,-0.095501,-0.105449,-0.115253,-0.124916,-0.134439,-0.143825,-0.153077,-0.162196,-0.171184,-0.180044,-0.188778,0.588934,0.000000,0.000000,0.0,0.0,0.000000,0.759759,0.0,0.284251,0.700437,0.012503
2,2018-06-01,2016-08-01,1.000000,0.944444,1.0,1.0,0.000000,1.353502,0,0,paid,55716,E,0.2499,36,0.274963,0.259472,0.244227,0.229225,0.214461,0.199930,0.185629,0.171554,0.157699,0.144063,0.130640,0.117426,0.104419,0.091615,0.079009,0.066600,0.054382,0.042352,0.030509,0.018847,0.007364,-0.003943,-0.015077,-0.026041,-0.036838,-0.047472,-0.057944,-0.068258,-0.078416,-0.088421,-0.098276,0.379164,0.000000,0.000000,0.0,0.0,0.175012,0.000000,0.0,0.240723,0.811421,0.063302
3,2011-06-01,2008-05-01,1.000000,1.000000,1.0,1.0,0.000000,1.173648,0,0,paid,55742,B,0.1071,36,0.087770,0.071643,0.055847,0.040373,0.025214,0.010362,-0.004190,-0.018448,-0.032420,-0.046112,-0.059532,-0.072685,-0.085577,-0.098215,-0.110604,-0.122751,-0.134660,-0.146337,-0.157788,-0.169018,-0.180031,-0.190833,-0.201429,-0.211822,-0.222018,-0.232022,-0.241836,-0.251467,-0.260917,-0.270191,-0.279292,0.140504,0.000000,0.247415,0.0,0.0,0.000000,0.000000,0.0,0.570257,0.860447,-0.000890
4,2018-04-01,2016-01-01,0.583209,1.000000,1.0,1.0,3677.225098,0.643185,1,1,charged_off,56121,A,0.0649,36,-0.385032,-0.390533,-0.395965,-0.401330,-0.406627,-0.411859,-0.417025,-0.422128,-0.427167,-0.432145,-0.437061,-0.441916,-0.446713,-0.451450,-0.456130,-0.460753,-0.465320,-0.469831,-0.474288,-0.478691,-0.483042,-0.487340,-0.491586,-0.495782,-0.499928,-0.504025,-0.508072,-0.512072,-0.516025,-0.519931,-0.523792,0.829386,0.162706,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.825928,0.852738,-0.025309


In [9]:
eval_loan_info.to_feather(os.path.join(dpath, 'eval_loan_info_scored.fth'))

# Catboost_regr_cbdp (catboost dataproc)

In [20]:
eval_loan_info = pd.read_feather(os.path.join(dpath, 'eval_loan_info_scored.fth'))
eval_loan_info.head()

,end_d,issue_d,maturity_paid,maturity_time,maturity_time_stat_adj,maturity_paid_stat_adj,rem_to_be_paid,roi_simple,target_loose,target_strict,loan_status,id,grade,int_rate,term,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.2,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.3,0.31,0.32,0.33,0.34,0.35,baseline_score,A_score,B_score,C_score,D_score,E_score,F_score,G_score,logistic_regr_score,catboost_clf_score,catboost_regr_score
0,2011-10-01,2009-08-01,1.000000,1.000000,1.0,1.0,0.000000,1.173214,0,0,paid,54734,B,0.1189,36,0.095902,0.081254,0.066865,0.052729,0.038841,0.025198,0.011792,-0.001379,-0.014321,-0.027038,-0.039534,-0.051815,-0.063884,-0.075745,-0.087403,-0.098861,-0.110124,-0.121195,-0.132078,-0.142776,-0.153294,-0.163634,-0.173800,-0.183796,-0.193624,-0.203289,-0.212792,-0.222138,-0.231329,-0.240368,-0.249259,0.989311,0.000000,0.812243,0.0,0.0,0.000000,0.000000,0.0,0.604838,0.895790,0.004597
1,2010-03-01,2008-07-01,1.000000,1.000000,1.0,1.0,0.000000,1.207769,0,0,paid,55521,F,0.1608,36,0.137690,0.124301,0.111112,0.098121,0.085324,0.072718,0.060299,0.048065,0.036013,0.024139,0.012441,0.000915,-0.010441,-0.021630,-0.032654,-0.043517,-0.054222,-0.064770,-0.075164,-0.085407,-0.095501,-0.105449,-0.115253,-0.124916,-0.134439,-0.143825,-0.153077,-0.162196,-0.171184,-0.180044,-0.188778,0.588934,0.000000,0.000000,0.0,0.0,0.000000,0.759759,0.0,0.284251,0.700437,0.012503
2,2018-06-01,2016-08-01,1.000000,0.944444,1.0,1.0,0.000000,1.353502,0,0,paid,55716,E,0.2499,36,0.274963,0.259472,0.244227,0.229225,0.214461,0.199930,0.185629,0.171554,0.157699,0.144063,0.130640,0.117426,0.104419,0.091615,0.079009,0.066600,0.054382,0.042352,0.030509,0.018847,0.007364,-0.003943,-0.015077,-0.026041,-0.036838,-0.047472,-0.057944,-0.068258,-0.078416,-0.088421,-0.098276,0.379164,0.000000,0.000000,0.0,0.0,0.175012,0.000000,0.0,0.240723,0.811421,0.063302
3,2011-06-01,2008-05-01,1.000000,1.000000,1.0,1.0,0.000000,1.173648,0,0,paid,55742,B,0.1071,36,0.087770,0.071643,0.055847,0.040373,0.025214,0.010362,-0.004190,-0.018448,-0.032420,-0.046112,-0.059532,-0.072685,-0.085577,-0.098215,-0.110604,-0.122751,-0.134660,-0.146337,-0.157788,-0.169018,-0.180031,-0.190833,-0.201429,-0.211822,-0.222018,-0.232022,-0.241836,-0.251467,-0.260917,-0.270191,-0.279292,0.140504,0.000000,0.247415,0.0,0.0,0.000000,0.000000,0.0,0.570257,0.860447,-0.000890
4,2018-04-01,2016-01-01,0.583209,1.000000,1.0,1.0,3677.225098,0.643185,1,1,charged_off,56121,A,0.0649,36,-0.385032,-0.390533,-0.395965,-0.401330,-0.406627,-0.411859,-0.417025,-0.422128,-0.427167,-0.432145,-0.437061,-0.441916,-0.446713,-0.451450,-0.456130,-0.460753,-0.465320,-0.469831,-0.474288,-0.478691,-0.483042,-0.487340,-0.491586,-0.495782,-0.499928,-0.504025,-0.508072,-0.512072,-0.516025,-0.519931,-0.523792,0.829386,0.162706,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.825928,0.852738,-0.025309


In [21]:
m = Model('catboost_regr_cbdp')
eval_loan_info['catboost_regr_cbdp_score'] = m.score(base_loan_info)

In [22]:
eval_loan_info.head()

,end_d,issue_d,maturity_paid,maturity_time,maturity_time_stat_adj,maturity_paid_stat_adj,rem_to_be_paid,roi_simple,target_loose,target_strict,loan_status,id,grade,int_rate,term,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.2,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.3,0.31,0.32,0.33,0.34,0.35,baseline_score,A_score,B_score,C_score,D_score,E_score,F_score,G_score,logistic_regr_score,catboost_clf_score,catboost_regr_score,catboost_regr_cbdp_score
0,2011-10-01,2009-08-01,1.000000,1.000000,1.0,1.0,0.000000,1.173214,0,0,paid,54734,B,0.1189,36,0.095902,0.081254,0.066865,0.052729,0.038841,0.025198,0.011792,-0.001379,-0.014321,-0.027038,-0.039534,-0.051815,-0.063884,-0.075745,-0.087403,-0.098861,-0.110124,-0.121195,-0.132078,-0.142776,-0.153294,-0.163634,-0.173800,-0.183796,-0.193624,-0.203289,-0.212792,-0.222138,-0.231329,-0.240368,-0.249259,0.989311,0.000000,0.812243,0.0,0.0,0.000000,0.000000,0.0,0.604838,0.895790,0.004597,0.021225
1,2010-03-01,2008-07-01,1.000000,1.000000,1.0,1.0,0.000000,1.207769,0,0,paid,55521,F,0.1608,36,0.137690,0.124301,0.111112,0.098121,0.085324,0.072718,0.060299,0.048065,0.036013,0.024139,0.012441,0.000915,-0.010441,-0.021630,-0.032654,-0.043517,-0.054222,-0.064770,-0.075164,-0.085407,-0.095501,-0.105449,-0.115253,-0.124916,-0.134439,-0.143825,-0.153077,-0.162196,-0.171184,-0.180044,-0.188778,0.588934,0.000000,0.000000,0.0,0.0,0.000000,0.759759,0.0,0.284251,0.700437,0.012503,0.004497
2,2018-06-01,2016-08-01,1.000000,0.944444,1.0,1.0,0.000000,1.353502,0,0,paid,55716,E,0.2499,36,0.274963,0.259472,0.244227,0.229225,0.214461,0.199930,0.185629,0.171554,0.157699,0.144063,0.130640,0.117426,0.104419,0.091615,0.079009,0.066600,0.054382,0.042352,0.030509,0.018847,0.007364,-0.003943,-0.015077,-0.026041,-0.036838,-0.047472,-0.057944,-0.068258,-0.078416,-0.088421,-0.098276,0.379164,0.000000,0.000000,0.0,0.0,0.175012,0.000000,0.0,0.240723,0.811421,0.063302,0.042187
3,2011-06-01,2008-05-01,1.000000,1.000000,1.0,1.0,0.000000,1.173648,0,0,paid,55742,B,0.1071,36,0.087770,0.071643,0.055847,0.040373,0.025214,0.010362,-0.004190,-0.018448,-0.032420,-0.046112,-0.059532,-0.072685,-0.085577,-0.098215,-0.110604,-0.122751,-0.134660,-0.146337,-0.157788,-0.169018,-0.180031,-0.190833,-0.201429,-0.211822,-0.222018,-0.232022,-0.241836,-0.251467,-0.260917,-0.270191,-0.279292,0.140504,0.000000,0.247415,0.0,0.0,0.000000,0.000000,0.0,0.570257,0.860447,-0.000890,0.014960
4,2018-04-01,2016-01-01,0.583209,1.000000,1.0,1.0,3677.225098,0.643185,1,1,charged_off,56121,A,0.0649,36,-0.385032,-0.390533,-0.395965,-0.401330,-0.406627,-0.411859,-0.417025,-0.422128,-0.427167,-0.432145,-0.437061,-0.441916,-0.446713,-0.451450,-0.456130,-0.460753,-0.465320,-0.469831,-0.474288,-0.478691,-0.483042,-0.487340,-0.491586,-0.495782,-0.499928,-0.504025,-0.508072,-0.512072,-0.516025,-0.519931,-0.523792,0.829386,0.162706,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.825928,0.852738,-0.025309,-0.044613


In [23]:
eval_loan_info.to_feather(os.path.join(dpath, 'eval_loan_info_scored.fth'))

# Catboost_clf_cbdp (catboost dataproc)

In [32]:
eval_loan_info = pd.read_feather(os.path.join(dpath, 'eval_loan_info_scored.fth'))
eval_loan_info.head()

,end_d,issue_d,maturity_paid,maturity_time,maturity_time_stat_adj,maturity_paid_stat_adj,rem_to_be_paid,roi_simple,target_loose,target_strict,loan_status,id,grade,int_rate,term,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.2,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.3,0.31,0.32,0.33,0.34,0.35,baseline_score,A_score,B_score,C_score,D_score,E_score,F_score,G_score,logistic_regr_score,catboost_clf_score,catboost_regr_score,catboost_regr_cbdp_score
0,2011-10-01,2009-08-01,1.000000,1.000000,1.0,1.0,0.000000,1.173214,0,0,paid,54734,B,0.1189,36,0.095902,0.081254,0.066865,0.052729,0.038841,0.025198,0.011792,-0.001379,-0.014321,-0.027038,-0.039534,-0.051815,-0.063884,-0.075745,-0.087403,-0.098861,-0.110124,-0.121195,-0.132078,-0.142776,-0.153294,-0.163634,-0.173800,-0.183796,-0.193624,-0.203289,-0.212792,-0.222138,-0.231329,-0.240368,-0.249259,0.989311,0.000000,0.812243,0.0,0.0,0.000000,0.000000,0.0,0.604838,0.895790,0.004597,0.021225
1,2010-03-01,2008-07-01,1.000000,1.000000,1.0,1.0,0.000000,1.207769,0,0,paid,55521,F,0.1608,36,0.137690,0.124301,0.111112,0.098121,0.085324,0.072718,0.060299,0.048065,0.036013,0.024139,0.012441,0.000915,-0.010441,-0.021630,-0.032654,-0.043517,-0.054222,-0.064770,-0.075164,-0.085407,-0.095501,-0.105449,-0.115253,-0.124916,-0.134439,-0.143825,-0.153077,-0.162196,-0.171184,-0.180044,-0.188778,0.588934,0.000000,0.000000,0.0,0.0,0.000000,0.759759,0.0,0.284251,0.700437,0.012503,0.004497
2,2018-06-01,2016-08-01,1.000000,0.944444,1.0,1.0,0.000000,1.353502,0,0,paid,55716,E,0.2499,36,0.274963,0.259472,0.244227,0.229225,0.214461,0.199930,0.185629,0.171554,0.157699,0.144063,0.130640,0.117426,0.104419,0.091615,0.079009,0.066600,0.054382,0.042352,0.030509,0.018847,0.007364,-0.003943,-0.015077,-0.026041,-0.036838,-0.047472,-0.057944,-0.068258,-0.078416,-0.088421,-0.098276,0.379164,0.000000,0.000000,0.0,0.0,0.175012,0.000000,0.0,0.240723,0.811421,0.063302,0.042187
3,2011-06-01,2008-05-01,1.000000,1.000000,1.0,1.0,0.000000,1.173648,0,0,paid,55742,B,0.1071,36,0.087770,0.071643,0.055847,0.040373,0.025214,0.010362,-0.004190,-0.018448,-0.032420,-0.046112,-0.059532,-0.072685,-0.085577,-0.098215,-0.110604,-0.122751,-0.134660,-0.146337,-0.157788,-0.169018,-0.180031,-0.190833,-0.201429,-0.211822,-0.222018,-0.232022,-0.241836,-0.251467,-0.260917,-0.270191,-0.279292,0.140504,0.000000,0.247415,0.0,0.0,0.000000,0.000000,0.0,0.570257,0.860447,-0.000890,0.014960
4,2018-04-01,2016-01-01,0.583209,1.000000,1.0,1.0,3677.225098,0.643185,1,1,charged_off,56121,A,0.0649,36,-0.385032,-0.390533,-0.395965,-0.401330,-0.406627,-0.411859,-0.417025,-0.422128,-0.427167,-0.432145,-0.437061,-0.441916,-0.446713,-0.451450,-0.456130,-0.460753,-0.465320,-0.469831,-0.474288,-0.478691,-0.483042,-0.487340,-0.491586,-0.495782,-0.499928,-0.504025,-0.508072,-0.512072,-0.516025,-0.519931,-0.523792,0.829386,0.162706,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.825928,0.852738,-0.025309,-0.044613


In [33]:
m = Model('catboost_clf_cbdp')
eval_loan_info['catboost_clf_cbdp_score'] = m.score(base_loan_info)

In [34]:
eval_loan_info.head()

,end_d,issue_d,maturity_paid,maturity_time,maturity_time_stat_adj,maturity_paid_stat_adj,rem_to_be_paid,roi_simple,target_loose,target_strict,loan_status,id,grade,int_rate,term,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.2,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.3,0.31,0.32,0.33,0.34,0.35,baseline_score,A_score,B_score,C_score,D_score,E_score,F_score,G_score,logistic_regr_score,catboost_clf_score,catboost_regr_score,catboost_regr_cbdp_score,catboost_clf_cbdp_score
0,2011-10-01,2009-08-01,1.000000,1.000000,1.0,1.0,0.000000,1.173214,0,0,paid,54734,B,0.1189,36,0.095902,0.081254,0.066865,0.052729,0.038841,0.025198,0.011792,-0.001379,-0.014321,-0.027038,-0.039534,-0.051815,-0.063884,-0.075745,-0.087403,-0.098861,-0.110124,-0.121195,-0.132078,-0.142776,-0.153294,-0.163634,-0.173800,-0.183796,-0.193624,-0.203289,-0.212792,-0.222138,-0.231329,-0.240368,-0.249259,0.989311,0.000000,0.812243,0.0,0.0,0.000000,0.000000,0.0,0.604838,0.895790,0.004597,0.021225,0.915779
1,2010-03-01,2008-07-01,1.000000,1.000000,1.0,1.0,0.000000,1.207769,0,0,paid,55521,F,0.1608,36,0.137690,0.124301,0.111112,0.098121,0.085324,0.072718,0.060299,0.048065,0.036013,0.024139,0.012441,0.000915,-0.010441,-0.021630,-0.032654,-0.043517,-0.054222,-0.064770,-0.075164,-0.085407,-0.095501,-0.105449,-0.115253,-0.124916,-0.134439,-0.143825,-0.153077,-0.162196,-0.171184,-0.180044,-0.188778,0.588934,0.000000,0.000000,0.0,0.0,0.000000,0.759759,0.0,0.284251,0.700437,0.012503,0.004497,0.742187
2,2018-06-01,2016-08-01,1.000000,0.944444,1.0,1.0,0.000000,1.353502,0,0,paid,55716,E,0.2499,36,0.274963,0.259472,0.244227,0.229225,0.214461,0.199930,0.185629,0.171554,0.157699,0.144063,0.130640,0.117426,0.104419,0.091615,0.079009,0.066600,0.054382,0.042352,0.030509,0.018847,0.007364,-0.003943,-0.015077,-0.026041,-0.036838,-0.047472,-0.057944,-0.068258,-0.078416,-0.088421,-0.098276,0.379164,0.000000,0.000000,0.0,0.0,0.175012,0.000000,0.0,0.240723,0.811421,0.063302,0.042187,0.594086
3,2011-06-01,2008-05-01,1.000000,1.000000,1.0,1.0,0.000000,1.173648,0,0,paid,55742,B,0.1071,36,0.087770,0.071643,0.055847,0.040373,0.025214,0.010362,-0.004190,-0.018448,-0.032420,-0.046112,-0.059532,-0.072685,-0.085577,-0.098215,-0.110604,-0.122751,-0.134660,-0.146337,-0.157788,-0.169018,-0.180031,-0.190833,-0.201429,-0.211822,-0.222018,-0.232022,-0.241836,-0.251467,-0.260917,-0.270191,-0.279292,0.140504,0.000000,0.247415,0.0,0.0,0.000000,0.000000,0.0,0.570257,0.860447,-0.000890,0.014960,0.882434
4,2018-04-01,2016-01-01,0.583209,1.000000,1.0,1.0,3677.225098,0.643185,1,1,charged_off,56121,A,0.0649,36,-0.385032,-0.390533,-0.395965,-0.401330,-0.406627,-0.411859,-0.417025,-0.422128,-0.427167,-0.432145,-0.437061,-0.441916,-0.446713,-0.451450,-0.456130,-0.460753,-0.465320,-0.469831,-0.474288,-0.478691,-0.483042,-0.487340,-0.491586,-0.495782,-0.499928,-0.504025,-0.508072,-0.512072,-0.516025,-0.519931,-0.523792,0.829386,0.162706,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.825928,0.852738,-0.025309,-0.044613,0.781861


In [35]:
eval_loan_info.to_feather(os.path.join(dpath, 'eval_loan_info_scored.fth'))